In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, MapType, DateType


# spark = SparkSession.builder \
#     .appName("Kafka to Minio") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("Kafka to Minio") \
    .config("spark.task.maxFailures", "1") \
    .config("fs.s3a.endpoint", "http://minio:9000") \
    .config("fs.s3a.access.key", "wFw2nWmmNHNivuqh") \
    .config("fs.s3a.secret.key", "Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs") \
    .config("fs.s3a.path.style.access", "true") \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.ssl.enabled", "false") \
    .getOrCreate()
# tiAoMHTLqpoi7TKMVXQGPR7N0VT6dRsJ
# E0QTNG3vCGYwqpar

spark.sparkContext.setLogLevel('INFO')

schema = StructType([
    StructField("id", IntegerType()),
    StructField("uid", StringType()),
    StructField("password", StringType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("username", StringType()),
    StructField("email", StringType()),
    StructField("avatar", StringType()),
    StructField("gender", StringType()),
    StructField("phone_number", StringType()),
    StructField("social_insurance_number", StringType()),
    StructField("date_of_birth", StringType()),
    StructField("employment", StructType([
        StructField("title", StringType()),
        StructField("key_skill", StringType())
    ])),
    StructField("address", StructType([
        StructField("city", StringType()),
        StructField("street_name", StringType()),
        StructField("street_address", StringType()),
        StructField("zip_code", StringType()),
        StructField("state", StringType()),
        StructField("country", StringType()),
        StructField("coordinates", StructType([
            StructField("lat", DoubleType()),
            StructField("lng", DoubleType())
        ]))
    ])),
    StructField("credit_card", StructType([
        StructField("cc_number", StringType())
    ])),
    StructField("subscription", StructType([
        StructField("plan", StringType()),
        StructField("status", StringType()),
        StructField("payment_method", StringType()),
        StructField("term", StringType())
    ]))
])
# spark.sparkContext.setLogLevel('ERROR')

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-broker:9092") \
    .option("subscribe", "user") \
    .load()

# df = df.selectExpr("CAST(value AS STRING)")

df = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data"))

# df.writeStream.outputMode("append").format("console").start().awaitTermination()


query = df \
    .writeStream \
    .format("json") \
    .option("path", "s3a://raw/user") \
    .option("checkpointLocation", "s3a://raw/checkpoint") \
    .start().awaitTermination()
# .trigger(processingTime='30 seconds') \.awaitTermination()
# query = df \
#     .writeStream \
#     .outputMode("append") \
#     .format("json") \
#     .option("path", "file:////home/user/kafka_test_t") \
#     .option("checkpointLocation", "file:////home/user/checkpoint") \
#     .start()

StreamingQueryException: Query [id = a5d2407e-ffc7-4261-8224-41b5a76b10ac, runId = fa18e56d-4176-450b-9c6d-e5d6ac90c208] terminated with exception: Job aborted.